In [ ]:
# Insurance RAG System using LangChain (Google Colab + Drive Version)

# --- Step 1: Install dependencies ---
!pip install -U langchain langchain-community faiss-cpu sentence-transformers pypdf chromadb gradio -q

# --- Step 2: Import required libraries ---
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import gradio as gr
import os

# --- Step 3: Mount Google Drive ---
from google.colab import drive
drive.mount('/content/drive')

# --- Step 4: Set up paths ---
docs_path = "/content/drive/MyDrive/insurance_docs"
index_path = "/content/drive/MyDrive/insurance_index"

# --- Step 5: Load insurance documents ---
if not os.path.exists(docs_path):
    os.makedirs(docs_path)
    print(f"Please upload insurance PDFs to: {docs_path}")

pdf_files = [os.path.join(docs_path, f) for f in os.listdir(docs_path) if f.endswith(".pdf")]
all_texts = []
for pdf in pdf_files:
    loader = PyPDFLoader(pdf)
    pages = loader.load()
    all_texts.extend(pages)

# --- Step 6: Split documents into chunks ---
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.split_documents(all_texts)

# --- Step 7: Create embeddings and FAISS index ---
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embedding_model)
db.save_local(index_path)

# --- Step 8: Create QA chain with Hugging Face LLM ---
hf_pipeline = pipeline("text2text-generation", model="google/flan-t5-base", max_length=512)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())

# --- Step 9: Define QA function for Gradio ---
def answer_question(question):
    return qa.run(question)

# --- Step 10: Launch Gradio chatbot ---
demo = gr.Interface(
    fn=answer_question,
    inputs=gr.Textbox(lines=2, placeholder="Ask about an insurance policy..."),
    outputs="text",
    title="Insurance RAG Chatbot",
    description="Ask questions based on uploaded insurance PDF documents."
)

demo.launch(share=True)